필요한 모듈 임포트

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
import platform
plt.rc('figure', figsize=(10, 6))  # 그림 크기 설정
if platform.system() == 'Windows': # 윈도우
    from matplotlib import font_manager, rc
    font_path = "C:/Windows/Fonts/NGULIM.TTF"
    font = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font)
elif platform.system() == 'Linux': # 우분투 또는 구글 코랩
    # !sudo apt-get install -y fonts-nanum*
    # !fc-cache -fv
    
    applyfont = "NanumBarunGothic"
    import matplotlib.font_manager as fm
    if not any(map(lambda ft: ft.name == applyfont, fm.fontManager.ttflist)):
        fm.fontManager.addfont("/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf")
    plt.rc("font", family=applyfont)
    plt.rc("axes", unicode_minus=False)

In [3]:
# GradientBoosting
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.preprocessing import StandardScaler

데이터 가져오기

In [5]:
base_path = "https://github.com/codingalzi/water-data/raw/master/reservoirs/"
# base_path = r"C:\Users\owner\Desktop\water-source\data"

In [7]:
scb_yearly = pd.read_excel(base_path+"Seungchonbo.xlsx", 
                            skiprows=0, 
                            header=1, 
                            sheet_name=None,
                            na_values='ND',
                            parse_dates=True)  # index_col = 0을 제외함

In [8]:
print(type(scb_yearly))  # type-dict
print(scb_yearly.keys())  # 2012년부터 2022년까지

<class 'dict'>
dict_keys(['2012년', '2013년', '2014년', '2015년', '2016년', '2017년', '2018년', '2019년', '2020년', '2021년', '2022년'])


In [9]:
scb_yearly['2012년'].columns

Index(['측정일/월', '측정소명', '수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥'],
      dtype='object')

2013년 자료부터

In [10]:
scb_yearly_df13 = pd.DataFrame()

for item in list(scb_yearly.keys())[1:]:
    scb_yearly_df13 = pd.concat([scb_yearly_df13, scb_yearly[item]])

In [11]:
scb_yearly_df13 = scb_yearly_df13.reset_index(drop=True)

결측치 확인

In [12]:
scb_yearly_df13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82257 entries, 0 to 82256
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   측정일/월        82257 non-null  object 
 1   측정소명         82257 non-null  object 
 2   수온 ℃         64574 non-null  float64
 3   수소이온농도       64561 non-null  float64
 4   전기전도도 μS/cm  64524 non-null  float64
 5   용존산소 mg/L    62656 non-null  float64
 6   총유기탄소 mg/L   58675 non-null  float64
 7   총질소 mg/L     59981 non-null  float64
 8   총인 mg/L      58997 non-null  float64
 9   클로로필-a mg/㎥  62876 non-null  float64
dtypes: float64(8), object(2)
memory usage: 6.3+ MB


In [13]:
scb_yearly_df13.isnull().sum()

측정일/월              0
측정소명               0
수온 ℃           17683
수소이온농도         17696
전기전도도 μS/cm    17733
용존산소 mg/L      19601
총유기탄소 mg/L     23582
총질소 mg/L       22276
총인 mg/L        23260
클로로필-a mg/㎥    19381
dtype: int64

In [14]:
scb_yearly_df13.head()

,측정일/월,측정소명,수온 ℃,수소이온농도,전기전도도 μS/cm,용존산소 mg/L,총유기탄소 mg/L,총질소 mg/L,총인 mg/L,클로로필-a mg/㎥
0,2013-01-01 00h,용봉,3.9,6.9,374.0,11.4,2.5,NaN,NaN,7.2
1,2013-01-01 01h,용봉,3.9,6.9,375.0,11.4,2.5,NaN,NaN,6.9
2,2013-01-01 02h,용봉,4.0,6.9,374.0,11.4,2.5,NaN,NaN,6.8
3,2013-01-01 03h,용봉,4.1,6.9,375.0,11.4,2.5,NaN,NaN,6.9
4,2013-01-01 04h,용봉,4.0,6.9,380.0,11.3,2.6,NaN,NaN,6.9


유량 데이터 추가하기

In [15]:
scb_flow_yearly = pd.read_excel(base_path+"Seungchonbo_flow.xlsx", 
                            header=0, 
                            sheet_name=None,
                            na_values='ND',
                            parse_dates=True)  # index_col = 0을 제외함

In [16]:
for year in list(scb_flow_yearly.keys())[1:]:
    scb_flow_yearly[year]['연월일시'] = scb_flow_yearly[year]['시간'].apply(lambda x : x[:-3])

In [17]:
scb_flow_yearly['2013']

,시간,상류수위(EL.m),하류수위(EL.m),유입량(㎥/s),방류량(㎥/s),연월일시
0,2013.01.01 00:00,7.50,3.77,19.82,19.74,2013.01.01 00
1,2013.01.01 00:10,7.50,3.77,19.78,19.76,2013.01.01 00
2,2013.01.01 00:20,7.50,3.77,19.77,19.82,2013.01.01 00
3,2013.01.01 00:30,7.50,3.77,19.77,19.73,2013.01.01 00
4,2013.01.01 00:40,7.50,3.77,19.78,19.79,2013.01.01 00
...,...,...,...,...,...,...
52555,2013.12.31 23:10,7.52,3.55,10.87,10.86,2013.12.31 23
52556,2013.12.31 23:20,7.52,3.55,10.87,10.87,2013.12.31 23
52557,2013.12.31 23:30,7.52,3.55,10.86,10.85,2013.12.31 23
52558,2013.12.31 23:40,7.52,3.55,10.86,10.85,2013.12.31 23


In [18]:
scb_flow_yearly_hour = {}

for year in list(scb_flow_yearly.keys())[1:]:
    scb_flow_yearly_hour[year] = scb_flow_yearly[year].groupby('연월일시').mean()

In [19]:
for year in scb_flow_yearly_hour:
    scb_flow_yearly_hour[year] = scb_flow_yearly_hour[year].reset_index()

In [20]:
scb_flow_yearly_hour['2013']

,연월일시,상류수위(EL.m),하류수위(EL.m),유입량(㎥/s),방류량(㎥/s)
0,2013.01.01 00,7.50,3.770000,19.773333,19.748333
1,2013.01.01 01,7.50,3.770000,19.701667,19.706667
2,2013.01.01 02,7.50,3.770000,19.748333,19.723333
3,2013.01.01 03,7.50,3.770000,19.613333,19.610000
4,2013.01.01 04,7.50,3.770000,19.661667,19.665000
...,...,...,...,...,...
8755,2013.12.31 19,7.52,3.558333,10.901667,10.910000
8756,2013.12.31 20,7.52,3.555000,10.845000,10.843333
8757,2013.12.31 21,7.52,3.551667,10.850000,10.845000
8758,2013.12.31 22,7.52,3.555000,10.870000,10.876667


In [21]:
scb_flow_yearly_hour_df = pd.DataFrame()

for year in scb_flow_yearly_hour:
    scb_flow_yearly_hour_df = pd.concat([scb_flow_yearly_hour_df, scb_flow_yearly_hour[year]])

In [22]:
scb_flow_yearly_hour_df = scb_flow_yearly_hour_df.reset_index(drop=True)

In [23]:
scb_flow_yearly_hour_df

,연월일시,상류수위(EL.m),하류수위(EL.m),유입량(㎥/s),방류량(㎥/s)
0,2013.01.01 00,7.500000,3.770000,19.773333,19.748333
1,2013.01.01 01,7.500000,3.770000,19.701667,19.706667
2,2013.01.01 02,7.500000,3.770000,19.748333,19.723333
3,2013.01.01 03,7.500000,3.770000,19.613333,19.610000
4,2013.01.01 04,7.500000,3.770000,19.661667,19.665000
...,...,...,...,...,...
84388,2022.08.17 15,5.536667,1.618333,29.261667,25.190000
84389,2022.08.17 16,5.525000,1.638333,23.243333,38.923333
84390,2022.08.17 17,5.483333,1.626667,29.068333,44.878333
84391,2022.08.17 18,5.470000,1.566667,19.131667,14.925000


In [24]:
print(scb_flow_yearly_hour_df.__len__())
print(scb_yearly_df13.__len__())

84393
82257


In [25]:
scb_yearly_df13['연월일시'] = scb_yearly_df13['측정일/월'].transform(lambda x: x.strip('h').replace('-', '.'))

In [26]:
scb_data = pd.merge(scb_yearly_df13, scb_flow_yearly_hour_df, how='outer')

In [27]:
scb_data = scb_data[:82257]

In [28]:
scb_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82257 entries, 0 to 82256
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   측정일/월        82257 non-null  object 
 1   측정소명         82257 non-null  object 
 2   수온 ℃         64574 non-null  float64
 3   수소이온농도       64561 non-null  float64
 4   전기전도도 μS/cm  64524 non-null  float64
 5   용존산소 mg/L    62656 non-null  float64
 6   총유기탄소 mg/L   58675 non-null  float64
 7   총질소 mg/L     59981 non-null  float64
 8   총인 mg/L      58997 non-null  float64
 9   클로로필-a mg/㎥  62876 non-null  float64
 10  연월일시         82257 non-null  object 
 11   상류수위(EL.m)  82231 non-null  float64
 12   하류수위(EL.m)  82231 non-null  float64
 13   유입량(㎥/s)    82231 non-null  float64
 14   방류량(㎥/s)    82231 non-null  float64
dtypes: float64(12), object(3)
memory usage: 10.0+ MB


In [29]:
scb_data.head()

,측정일/월,측정소명,수온 ℃,수소이온농도,전기전도도 μS/cm,용존산소 mg/L,총유기탄소 mg/L,총질소 mg/L,총인 mg/L,클로로필-a mg/㎥,연월일시,상류수위(EL.m),하류수위(EL.m),유입량(㎥/s),방류량(㎥/s)
0,2013-01-01 00h,용봉,3.9,6.9,374.0,11.4,2.5,NaN,NaN,7.2,2013.01.01 00,7.5,3.77,19.773333,19.748333
1,2013-01-01 01h,용봉,3.9,6.9,375.0,11.4,2.5,NaN,NaN,6.9,2013.01.01 01,7.5,3.77,19.701667,19.706667
2,2013-01-01 02h,용봉,4.0,6.9,374.0,11.4,2.5,NaN,NaN,6.8,2013.01.01 02,7.5,3.77,19.748333,19.723333
3,2013-01-01 03h,용봉,4.1,6.9,375.0,11.4,2.5,NaN,NaN,6.9,2013.01.01 03,7.5,3.77,19.613333,19.610000
4,2013-01-01 04h,용봉,4.0,6.9,380.0,11.3,2.6,NaN,NaN,6.9,2013.01.01 04,7.5,3.77,19.661667,19.665000


In [30]:
scb_data.columns = [x.strip() for x in list(scb_data.columns)]

In [31]:
scb_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82257 entries, 0 to 82256
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   측정일/월        82257 non-null  object 
 1   측정소명         82257 non-null  object 
 2   수온 ℃         64574 non-null  float64
 3   수소이온농도       64561 non-null  float64
 4   전기전도도 μS/cm  64524 non-null  float64
 5   용존산소 mg/L    62656 non-null  float64
 6   총유기탄소 mg/L   58675 non-null  float64
 7   총질소 mg/L     59981 non-null  float64
 8   총인 mg/L      58997 non-null  float64
 9   클로로필-a mg/㎥  62876 non-null  float64
 10  연월일시         82257 non-null  object 
 11  상류수위(EL.m)   82231 non-null  float64
 12  하류수위(EL.m)   82231 non-null  float64
 13  유입량(㎥/s)     82231 non-null  float64
 14  방류량(㎥/s)     82231 non-null  float64
dtypes: float64(12), object(3)
memory usage: 10.0+ MB


In [32]:
scb_data = scb_data.loc[:, list(scb_data.columns)[:-5] + list(scb_data.columns)[-4:-3] + list(scb_data.columns)[-1:]]

In [33]:
scb_data.columns

Index(['측정일/월', '측정소명', '수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥', '상류수위(EL.m)',
       '방류량(㎥/s)'],
      dtype='object')

In [34]:
scb_data

,측정일/월,측정소명,수온 ℃,수소이온농도,전기전도도 μS/cm,용존산소 mg/L,총유기탄소 mg/L,총질소 mg/L,총인 mg/L,클로로필-a mg/㎥,상류수위(EL.m),방류량(㎥/s)
0,2013-01-01 00h,용봉,3.9,6.9,374.0,11.4,2.5,NaN,NaN,7.2,7.50,19.748333
1,2013-01-01 01h,용봉,3.9,6.9,375.0,11.4,2.5,NaN,NaN,6.9,7.50,19.706667
2,2013-01-01 02h,용봉,4.0,6.9,374.0,11.4,2.5,NaN,NaN,6.8,7.50,19.723333
3,2013-01-01 03h,용봉,4.1,6.9,375.0,11.4,2.5,NaN,NaN,6.9,7.50,19.610000
4,2013-01-01 04h,용봉,4.0,6.9,380.0,11.3,2.6,NaN,NaN,6.9,7.50,19.665000
...,...,...,...,...,...,...,...,...,...,...,...,...
82252,2022-05-23 19h,용봉,26.6,6.9,419.0,NaN,6.7,5.825,0.108,66.2,5.44,12.766667
82253,2022-05-23 20h,용봉,26.3,6.8,425.0,NaN,6.5,5.803,0.107,61.2,5.44,12.786667
82254,2022-05-23 21h,용봉,25.9,6.8,417.0,NaN,6.7,5.909,0.107,58.3,5.44,12.743333
82255,2022-05-23 22h,용봉,25.7,6.8,420.0,NaN,6.4,5.921,0.106,54.4,5.44,12.700000


In [35]:
scb_data.isnull().sum()

측정일/월              0
측정소명               0
수온 ℃           17683
수소이온농도         17696
전기전도도 μS/cm    17733
용존산소 mg/L      19601
총유기탄소 mg/L     23582
총질소 mg/L       22276
총인 mg/L        23260
클로로필-a mg/㎥    19381
상류수위(EL.m)        26
방류량(㎥/s)          26
dtype: int64

특성별 결치비 비율

In [36]:
scb_data.isnull().sum() /len(scb_data)

측정일/월          0.000000
측정소명           0.000000
수온 ℃           0.214973
수소이온농도         0.215131
전기전도도 μS/cm    0.215580
용존산소 mg/L      0.238290
총유기탄소 mg/L     0.286687
총질소 mg/L       0.270810
총인 mg/L        0.282772
클로로필-a mg/㎥    0.235615
상류수위(EL.m)     0.000316
방류량(㎥/s)       0.000316
dtype: float64

In [36]:
len(scb_data)

82257

시간데이터 추가

In [37]:
scb_df = scb_data.copy()

In [38]:
scb_df.columns

Index(['측정일/월', '측정소명', '수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L',
       '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥', '상류수위(EL.m)',
       '방류량(㎥/s)'],
      dtype='object')

In [39]:
col_c = list(scb_df.columns.copy())
col_c.remove('클로로필-a mg/㎥')

In [40]:
col_c

['측정일/월',
 '측정소명',
 '수온 ℃',
 '수소이온농도',
 '전기전도도 μS/cm',
 '용존산소 mg/L',
 '총유기탄소 mg/L',
 '총질소 mg/L',
 '총인 mg/L',
 '상류수위(EL.m)',
 '방류량(㎥/s)']

결측치 보간법

In [41]:
from datetime import datetime

In [42]:
scb_df1 = scb_df.copy()

In [43]:
scb_df1['측정일/월'] = scb_df1['측정일/월'].transform(lambda x: datetime.strptime(x[:-1], '%Y-%m-%d %H'))

In [44]:
scb_df1

,측정일/월,측정소명,수온 ℃,수소이온농도,전기전도도 μS/cm,용존산소 mg/L,총유기탄소 mg/L,총질소 mg/L,총인 mg/L,클로로필-a mg/㎥,상류수위(EL.m),방류량(㎥/s)
0,2013-01-01 00:00:00,용봉,3.9,6.9,374.0,11.4,2.5,NaN,NaN,7.2,7.50,19.748333
1,2013-01-01 01:00:00,용봉,3.9,6.9,375.0,11.4,2.5,NaN,NaN,6.9,7.50,19.706667
2,2013-01-01 02:00:00,용봉,4.0,6.9,374.0,11.4,2.5,NaN,NaN,6.8,7.50,19.723333
3,2013-01-01 03:00:00,용봉,4.1,6.9,375.0,11.4,2.5,NaN,NaN,6.9,7.50,19.610000
4,2013-01-01 04:00:00,용봉,4.0,6.9,380.0,11.3,2.6,NaN,NaN,6.9,7.50,19.665000
...,...,...,...,...,...,...,...,...,...,...,...,...
82252,2022-05-23 19:00:00,용봉,26.6,6.9,419.0,NaN,6.7,5.825,0.108,66.2,5.44,12.766667
82253,2022-05-23 20:00:00,용봉,26.3,6.8,425.0,NaN,6.5,5.803,0.107,61.2,5.44,12.786667
82254,2022-05-23 21:00:00,용봉,25.9,6.8,417.0,NaN,6.7,5.909,0.107,58.3,5.44,12.743333
82255,2022-05-23 22:00:00,용봉,25.7,6.8,420.0,NaN,6.4,5.921,0.106,54.4,5.44,12.700000


In [45]:
scb_df1 = scb_df1.copy().set_index('측정일/월').interpolate(method='time')

In [46]:
scb_df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82257 entries, 2013-01-01 00:00:00 to 2022-05-23 23:00:00
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   측정소명         82257 non-null  object 
 1   수온 ℃         82257 non-null  float64
 2   수소이온농도       82257 non-null  float64
 3   전기전도도 μS/cm  82257 non-null  float64
 4   용존산소 mg/L    82257 non-null  float64
 5   총유기탄소 mg/L   82257 non-null  float64
 6   총질소 mg/L     80154 non-null  float64
 7   총인 mg/L      80154 non-null  float64
 8   클로로필-a mg/㎥  82257 non-null  float64
 9   상류수위(EL.m)   82257 non-null  float64
 10  방류량(㎥/s)     82257 non-null  float64
dtypes: float64(10), object(1)
memory usage: 7.5+ MB


In [47]:
scb_df1 = scb_df1.dropna()

In [50]:
scb_df1.__len__()

80154

In [51]:
scb_df1.columns

Index(['측정소명', '수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L', '총유기탄소 mg/L',
       '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥', '상류수위(EL.m)', '방류량(㎥/s)'],
      dtype='object')

In [52]:
col = list(scb_df1.columns[1:])
print(col)

['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L', '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '클로로필-a mg/㎥', '상류수위(EL.m)', '방류량(㎥/s)']


In [53]:
target_feature = '클로로필-a mg/㎥'

In [54]:
col_v = col.copy()
col_v.remove(target_feature)
print(col_v)

['수온 ℃', '수소이온농도', '전기전도도 μS/cm', '용존산소 mg/L', '총유기탄소 mg/L', '총질소 mg/L', '총인 mg/L', '상류수위(EL.m)', '방류량(㎥/s)']


**시계열 데이터로 전처리**

**피처 정규화 없이**

날짜별로 정렬

In [55]:
scb_df1 = scb_df1.sort_index()

훈련셋, 검증셋, 테스트셋 지정

In [56]:
train_size = int(scb_df1.shape[0] * 0.7)
val_size = int(scb_df1.shape[0] * 0.2)

- 훈련셋

In [57]:
train_set = scb_df1[col_v][:train_size]
train_targets = scb_df1[target_feature][:train_size]

- 검증셋

In [58]:
val_set = scb_df1[col_v][train_size : train_size+val_size]
val_targets = scb_df1[target_feature][train_size : train_size+val_size]

- 검증셋

In [59]:
test_set = scb_df1[col_v][train_size+val_size:]
test_targets = scb_df1[target_feature][train_size+val_size:]

시계열 데이터로 변환

In [60]:
import tensorflow as tf

타임스텝 크기: 72

시계열 데이터 샘플을 `sequence_length` 만큼의 타임 스텝(time step)로
구성한다.
예측값은 미래가 아닌 현재의 클로로필-A 수치로 지정한다.

- 배치 크기: 32
- 공정한 훈련을 위해 구성된 시계열 샘플을 뒤섞는다.

In [60]:
sequence_length=72  # 타임 스텝 크기

train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    train_set,
    targets=train_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

val_dataset = tf.keras.utils.timeseries_dataset_from_array(
    val_set,
    targets=val_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    test_set,
    targets=test_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=32)

입력 데이터셋의 배치의 모양은 `(32, 72, 9)`이다.

In [61]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

samples shape: (32, 72, 9)
targets shape: (32,)


**케라스 활용**

In [62]:
from tensorflow import keras
from tensorflow.keras import layers

**`Dense` 모델 사용**

타임시리즈로 구성된 샘플을 1차원 텐서로 푼 다음에 `Dense` 모델을 적용해본다.

In [65]:
dropout_rate = 0.5

inputs = keras.Input(shape=(sequence_length, len(col)-1))
x = layers.Flatten()(inputs)
x = layers.Dense(1024, activation="relu")(x)
x = layers.Dropout(dropout_rate)(x)
# x = layers.Dense(1024, activation="relu")(x)
# x = layers.Dropout(dropout_rate)(x)
# x = layers.Dense(1024, activation="relu")(x)
# x = layers.Dropout(dropout_rate)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("seuncheonbo_dense.keras",
                                    save_best_only=True)
] 

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("seuncheonbo_dense.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
1752/1752 [==============================] - 6s 3ms/step - loss: 4368.9268 - mae: 30.0275 - val_loss: 1585.4474 - val_mae: 29.7668
Epoch 2/20
1752/1752 [==============================] - 5s 3ms/step - loss: 771.1382 - mae: 20.2383 - val_loss: 1057.6534 - val_mae: 24.3976
Epoch 3/20
1752/1752 [==============================] - 5s 3ms/step - loss: 736.4756 - mae: 19.6480 - val_loss: 1993.4453 - val_mae: 32.7304
Epoch 4/20
1752/1752 [==============================] - 5s 3ms/step - loss: 734.2389 - mae: 19.6208 - val_loss: 1006.7623 - val_mae: 25.8506
Epoch 5/20
1752/1752 [==============================] - 5s 3ms/step - loss: 722.6017 - mae: 19.4033 - val_loss: 938.1530 - val_mae: 23.9578
Epoch 6/20
1752/1752 [==============================] - 5s 3ms/step - loss: 715.3601 - mae: 19.3233 - val_loss: 2150.5527 - val_mae: 34.4925
Epoch 7/20
1752/1752 [==============================] - 5s 3ms/step - loss: 714.1165 - mae: 19.2352 - val_loss: 958.8087 - val_mae: 23.6666
Epoch 8/20
175

**피처 정규화 활용**

In [68]:
scb_df2 = scb_df1.copy()
scb_df2 = scb_df2[col]

In [70]:
scb_df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 80154 entries, 2013-03-29 16:00:00 to 2022-05-23 23:00:00
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   수온 ℃         80154 non-null  float64
 1   수소이온농도       80154 non-null  float64
 2   전기전도도 μS/cm  80154 non-null  float64
 3   용존산소 mg/L    80154 non-null  float64
 4   총유기탄소 mg/L   80154 non-null  float64
 5   총질소 mg/L     80154 non-null  float64
 6   총인 mg/L      80154 non-null  float64
 7   클로로필-a mg/㎥  80154 non-null  float64
 8   상류수위(EL.m)   80154 non-null  float64
 9   방류량(㎥/s)     80154 non-null  float64
dtypes: float64(10)
memory usage: 6.7 MB


모든 특성을 정규화 한다.

먼저 특성별 평균값과 표준편차를 계산한다.

In [71]:
scb_df2_mean = scb_df2.mean(axis=0)
scb_df2_std = scb_df2.std(axis=0)

In [72]:
scb_df2 = (scb_df2 - scb_df2_mean)/scb_df2_std
scb_df2.describe()

,수온 ℃,수소이온농도,전기전도도 μS/cm,용존산소 mg/L,총유기탄소 mg/L,총질소 mg/L,총인 mg/L,클로로필-a mg/㎥,상류수위(EL.m),방류량(㎥/s)
count,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04,8.015400e+04
mean,-1.418355e-17,6.410965e-16,-5.503218e-16,1.418355e-16,-1.049583e-16,-2.666508e-16,-9.148391e-17,1.928963e-16,1.242479e-15,4.396901e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.054561e+00,-1.758730e+00,-2.860171e+00,-1.737012e+00,-2.877265e+00,-2.393871e+00,-1.857785e+00,-1.338400e+00,-2.485170e+00,-2.820256e-01
25%,-8.650653e-01,-5.698208e-01,-7.036627e-01,-6.763458e-01,-6.836776e-01,-7.798853e-01,-7.275951e-01,-7.565927e-01,-5.613570e-01,-1.847408e-01
50%,-8.078240e-02,-1.735176e-01,7.093631e-02,-1.725295e-01,-1.604366e-01,-1.190685e-01,-1.698392e-01,-2.419173e-01,5.240373e-01,-1.503697e-01
75%,9.126427e-01,2.227855e-01,7.831518e-01,7.025199e-01,7.451728e-01,6.733068e-01,4.759834e-01,5.580673e-01,7.125189e-01,-4.785549e-02
max,2.428923e+00,5.969182e+00,4.381075e+00,4.573951e+00,1.493305e+01,3.478880e+00,9.033133e+00,1.131870e+01,9.846300e+01,3.555074e+01


훈련셋, 검증셋, 테스트셋 지정

In [73]:
train_size = int(scb_df2.shape[0] * 0.7)
val_size = int(scb_df2.shape[0] * 0.2)

- 훈련셋

In [74]:
train_set = scb_df2[col_v][:train_size]
train_targets = scb_df2[target_feature][:train_size]

- 검증셋

In [75]:
val_set = scb_df2[col_v][train_size : train_size+val_size]
val_targets = scb_df2[target_feature][train_size : train_size+val_size]

- 검증셋

In [76]:
test_set = scb_df2[col_v][train_size+val_size:]
test_targets = scb_df2[target_feature][train_size+val_size:]

시계열 데이터로 변환

In [77]:
import tensorflow as tf

시계열 데이터 샘플을 `sequence_length` 만큼의 타임 스텝(time step)로
구성한다.
예측값은 미래가 아닌 현재의 클로로필-A 수치로 지정한다.

- 배치 크기: 32
- 공정한 훈련을 위해 구성된 시계열 샘플을 뒤섞는다.

In [99]:
sequence_length=72  # 타임 스텝 크기
batch_size = 1024

train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    train_set,
    targets=train_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size)

val_dataset = tf.keras.utils.timeseries_dataset_from_array(
    val_set,
    targets=val_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size)

test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    test_set,
    targets=test_targets[sequence_length-1:],
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size)

입력 데이터셋의 배치의 모양은 `(32, 72, 9)`이다.

In [100]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    break

samples shape: (1024, 72, 9)
targets shape: (1024,)


**케라스 활용**

In [80]:
from tensorflow import keras
from tensorflow.keras import layers

**`Dense` 모델 사용**

타임시리즈로 구성된 샘플을 1차원 텐서로 푼 다음에 `Dense` 모델을 적용해본다.

In [104]:
dropout_rate = 0.5

inputs = keras.Input(shape=(sequence_length, len(col)-1))
x = layers.Flatten()(inputs)
x = layers.Dense(18, activation="relu")(x)
x = layers.Dense(18, activation="relu")(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

callbacks = [
    keras.callbacks.ModelCheckpoint("seuncheonbo_dense.keras",
                                    save_best_only=True)
] 

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("seuncheonbo_dense.keras")
print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
55/55 [==============================] - 2s 21ms/step - loss: 0.4080 - mae: 0.4536 - val_loss: 0.8900 - val_mae: 0.7703
Epoch 2/20
55/55 [==============================] - 1s 21ms/step - loss: 0.3065 - mae: 0.3825 - val_loss: 1.0231 - val_mae: 0.8376
Epoch 3/20
55/55 [==============================] - 1s 20ms/step - loss: 0.2729 - mae: 0.3589 - val_loss: 1.0667 - val_mae: 0.8426
Epoch 4/20
55/55 [==============================] - 1s 20ms/step - loss: 0.2509 - mae: 0.3472 - val_loss: 1.0161 - val_mae: 0.8061
Epoch 5/20
55/55 [==============================] - 1s 19ms/step - loss: 0.2376 - mae: 0.3377 - val_loss: 0.9779 - val_mae: 0.7980
Epoch 6/20
55/55 [==============================] - 1s 20ms/step - loss: 0.2255 - mae: 0.3299 - val_loss: 1.1098 - val_mae: 0.8593
Epoch 7/20
55/55 [==============================] - 1s 21ms/step - loss: 0.2151 - mae: 0.3223 - val_loss: 1.1094 - val_mae: 0.8546
Epoch 8/20
55/55 [==============================] - 1s 20ms/step - loss: 0.2068 - m

**GRU 모델 활용**

In [110]:
inputs = keras.Input(shape=(sequence_length, len(col)-1))
x = layers.GRU(18, recurrent_dropout=0.5, return_sequences=True)(inputs)
x = layers.GRU(18, recurrent_dropout=0.5)(x)
# x = layers.GRU(9, recurrent_dropout=0.5)(inputs)
# x = layers.GRU(128, recurrent_dropout=0.5, return_sequences=True)(inputs)
# x = layers.GRU(128, recurrent_dropout=0.5, return_sequences=True)(x)
# x = layers.GRU(128, recurrent_dropout=0.5)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
  
callbacks = [
    keras.callbacks.ModelCheckpoint("yeongsan_gru.keras",
                                    save_best_only=True)
]
                     
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=val_dataset,
                    callbacks=callbacks)

model = keras.models.load_model("yeongsan_gru.keras") 

print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

Epoch 1/20
55/55 [==============================] - 30s 489ms/step - loss: 0.5989 - mae: 0.5826 - val_loss: 0.7656 - val_mae: 0.6739
Epoch 2/20
55/55 [==============================] - 28s 504ms/step - loss: 0.4522 - mae: 0.4922 - val_loss: 0.7105 - val_mae: 0.6358
Epoch 3/20
55/55 [==============================] - 27s 493ms/step - loss: 0.4085 - mae: 0.4619 - val_loss: 0.7043 - val_mae: 0.6303
Epoch 4/20
55/55 [==============================] - 27s 492ms/step - loss: 0.3860 - mae: 0.4444 - val_loss: 0.7064 - val_mae: 0.6314
Epoch 5/20
55/55 [==============================] - 27s 492ms/step - loss: 0.3694 - mae: 0.4316 - val_loss: 0.7259 - val_mae: 0.6390
Epoch 6/20
17/55 [========>.....................] - ETA: 17s - loss: 0.3627 - mae: 0.4300

KeyboardInterrupt: 